Importing Libraries

In [1]:
import pandas as pd 
import wikipedia as wp
import requests
from bs4 import BeautifulSoup


Loading data from url to dataframe

In [2]:
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")

# determine the index of your table
df = pd.read_html(html, header = 0)[0]


Only process cells that have borough, if more than one neighborhood are present in a postal code, then combine them and if a cell has a borough but no neighbourhood, then the neighbourhood is the borough

In [3]:
# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df = df[df.Borough != 'Not assigned']

# More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
df = df.groupby('Postal code').agg(
    {
        'Borough':'first', 
        'Neighborhood': ', '.join,}
    ).reset_index()
df.head(10)

# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
for index, row in df.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] = row['Borough']

Lets see how our dataframe looks:

In [4]:
df

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park
7,M1L,Scarborough,Golden Mile / Clairlea / Oakridge
8,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village West
9,M1N,Scarborough,Birch Cliff / Cliffside West


 In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe



In [5]:
# In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
df.shape


(103, 3)

loading geospatial data into data frame and merging


In [16]:
import io
url="http://cocl.us/Geospatial_data"
r=requests.get(url).content
dg= pd.read_csv(io.StringIO(r.decode('utf-8')))

# rename the first column to allow merging dataframes on Postcode
dg.columns = ['Postal code', 'Latitude', 'Longitude']
df = pd.merge(dg, df, on='Postal code')

df

,Postal code,Latitude,Longitude,Latitude_x,Longitude_x,Latitude_y,Longitude_y,Borough,Neighborhood
0,M1B,43.806686,-79.194353,43.806686,-79.194353,43.806686,-79.194353,Scarborough,Malvern / Rouge
1,M1C,43.784535,-79.160497,43.784535,-79.160497,43.784535,-79.160497,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,43.763573,-79.188711,43.763573,-79.188711,43.763573,-79.188711,Scarborough,Guildwood / Morningside / West Hill
3,M1G,43.770992,-79.216917,43.770992,-79.216917,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,43.773136,-79.239476,43.773136,-79.239476,Scarborough,Cedarbrae
5,M1J,43.744734,-79.239476,43.744734,-79.239476,43.744734,-79.239476,Scarborough,Scarborough Village
6,M1K,43.727929,-79.262029,43.727929,-79.262029,43.727929,-79.262029,Scarborough,Kennedy Park / Ionview / East Birchmount Park
7,M1L,43.711112,-79.284577,43.711112,-79.284577,43.711112,-79.284577,Scarborough,Golden Mile / Clairlea / Oakridge
8,M1M,43.716316,-79.239476,43.716316,-79.239476,43.716316,-79.239476,Scarborough,Cliffside / Cliffcrest / Scarborough Village West
9,M1N,43.692657,-79.264848,43.692657,-79.264848,43.692657,-79.264848,Scarborough,Birch Cliff / Cliffside West


Part 3: Map analysis

In [7]:
#importing relevant modules
import matplotlib.pyplot as plt
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
print('Imported successfully')

Solving environment: ...working... done

# All requested packages already installed.

Imported successfully


Accessing Latitude and Longitude

In [9]:
address = 'Toronto'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [10]:
import folium # map rendering library
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(df, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

Saving FourSquare Credentials 

In [11]:
CLIENT_ID = 'ODCSRO0KD4YAUTVVNDBXAG0GQAHHMNWC4QQUGYGPJVFKDZBK' # your Foursquare ID
CLIENT_SECRET = 'UVHSLY4BFWD51I1BVGWMBR3U3LM32LTCULH1ZYQHHP0KP312'  # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ODCSRO0KD4YAUTVVNDBXAG0GQAHHMNWC4QQUGYGPJVFKDZBK
CLIENT_SECRET:UVHSLY4BFWD51I1BVGWMBR3U3LM32LTCULH1ZYQHHP0KP312


Lets Explore first neighborhood in the dataset

In [21]:
borough_names = list(df.Borough.unique()) #Using only the Boroughs that contain Toronto

borough_with_toronto = []

for x in borough_names:
    if "toronto" in x.lower():
        borough_with_toronto.append(x)
        
borough_with_toronto

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']

Create a new dataframe which only has borighs in toronto

In [25]:
df1 = df[df['Borough'].isin(borough_with_toronto)].reset_index(drop=True)
print(df1.shape)
df1.head()

(39, 9)


,Postal code,Latitude,Longitude,Latitude_x,Longitude_x,Latitude_y,Longitude_y,Borough,Neighborhood
0,M4E,43.676357,-79.293031,43.676357,-79.293031,43.676357,-79.293031,East Toronto,The Beaches
1,M4K,43.679557,-79.352188,43.679557,-79.352188,43.679557,-79.352188,East Toronto,The Danforth West / Riverdale
2,M4L,43.668999,-79.315572,43.668999,-79.315572,43.668999,-79.315572,East Toronto,India Bazaar / The Beaches West
3,M4M,43.659526,-79.340923,43.659526,-79.340923,43.659526,-79.340923,East Toronto,Studio District
4,M4N,43.728020,-79.388790,43.728020,-79.388790,43.728020,-79.388790,Central Toronto,Lawrence Park


In [27]:
#Drop Unnecessary columns
df1.drop(columns=['Latitude_x', 'Latitude_y','Longitude_x', 'Longitude_y'])
# Re-order Columns
df = df[['Postal code','Borough','Neighborhood','Latitude','Longitude']]
df.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [28]:
df1.loc[0, 'Neighborhood'] #First neighbourhood in our datadrame

'The Beaches'

In [29]:
#Latitude and Longitude Values of the first neighborhood
neighborhood_latitude = df1.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df1.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df1.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


In [ ]:
Lets get the top 50 venues that are nearby The Beaches in a radius of 500

In [30]:
LIMIT = 50 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=ODCSRO0KD4YAUTVVNDBXAG0GQAHHMNWC4QQUGYGPJVFKDZBK&client_secret=UVHSLY4BFWD51I1BVGWMBR3U3LM32LTCULH1ZYQHHP0KP312&v=20180605&ll=43.67635739999999,-79.2930312&radius=500&limit=50'

Get Results in Json format

In [31]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ea45db4006dce001b3d0346'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.680857404499996,
    'lng': -79.28682091449052},
   'sw': {'lat': 43.67185739549999, 'lng': -79.29924148550948}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'labe

In [35]:
           
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [39]:
toronto_venues = getNearbyVenues(names=df1['Neighborhood'],
                                   latitudes=df1['Latitude'],
                                   longitudes=df1['Longitude']
                                  )


The Beaches
The Danforth West / Riverdale
India Bazaar / The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park / Summerhill East
Summerhill West / Rathnelly / South Hill / Forest Hill SE / Deer Park
Rosedale
St. James Town / Cabbagetown
Church and Wellesley
Regent Park / Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond / Adelaide / King
Harbourfront East / Union Station / Toronto Islands
Toronto Dominion Centre / Design Exchange
Commerce Court / Victoria Hotel
Roselawn
Forest Hill North & West
The Annex / North Midtown / Yorkville
University of Toronto / Harbord
Kensington Market / Chinatown / Grange Park
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport
Stn A PO Boxes
First Canadian Place / Underground city
Christie
Dufferin / Dovercourt Village
Little Portugal / Trinity
Brockton / Parkdale Village / Exhibition Place
High Park / 

In [40]:
print(toronto_venues.shape)
toronto_venues.head()

(1190, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Danforth West / Riverdale,43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


In [41]:
#Checking the count of neighbourhoods
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,50,50,50,50,50,50
Brockton / Parkdale Village / Exhibition Place,22,22,22,22,22,22
Business reply mail Processing CentrE,18,18,18,18,18,18
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport,18,18,18,18,18,18
Central Bay Street,50,50,50,50,50,50
Christie,18,18,18,18,18,18
Church and Wellesley,50,50,50,50,50,50
Commerce Court / Victoria Hotel,50,50,50,50,50,50
Davisville,33,33,33,33,33,33


Checking out unique categories of venues

In [44]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 213 uniques categories.


Analysing each neighbourhood

In [60]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")


toronto_onehot['Neighborhoods'] = toronto_venues['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head(20)

(1190, 214)


,Wine Shop,Yoga Studio,Neighborhoods,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,0,0,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0,0,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,The Danforth West / Riverdale,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,The Danforth West / Riverdale,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,The Danforth West / Riverdale,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,The Danforth West / Riverdale,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,The Danforth West / Riverdale,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,1,The Danforth West / Riverdale,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [63]:
toronto_onehot.shape

(1190, 214)

In [66]:

toronto_grouped = toronto_onehot.groupby([ "Neighborhoods"]).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped


(39, 214)


,Neighborhoods,Wine Shop,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.020000,0.000000,0.00,0.000000,0.000000,0.00,0.020000,0.000000,0.00000,0.00000
1,Brockton / Parkdale Village / Exhibition Place,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00000,0.00000
2,Business reply mail Processing CentrE,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00000,0.00000
3,CN Tower / King and Spadina / Railway Lands / ...,0.000000,0.000000,0.055556,0.055556,0.055556,0.111111,0.166667,0.111111,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00000,0.00000
4,Central Bay Street,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.020000,0.000000,0.00000,0.00000
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00000,0.00000
6,Church and Wellesley,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.020000,0.000000,0.02,0.000000,0.000000,0.00,0.000000,0.000000,0.00000,0.00000
7,Commerce Court / Victoria Hotel,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00000,0.02000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.030303,0.000000,0.00,0.030303,0.000000,0.00,0.000000,0.000000,0.00000,0.00000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00000,0.00000


In [67]:
toronto_grouped.shape

(39, 214)

CLUSTERING NEIGHBOURHOODS 

In [72]:
#Running K means clustering to make 5 clusters
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])